# Building and simulating a Protein Simulation to produce trajectory files for pi-stacking analysis

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import random
import pandas as pd
import time
import seaborn as sns

import mdtraj as md
import nglview
import openmm
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path
from openff.toolkit.utils.toolkits import RDKitToolkitWrapper
from openff.units import unit
from pandas import read_csv
import espaloma_charge as espcharge
from espaloma_charge.openff_wrapper import EspalomaChargeToolkitWrapper
from pdbfixer import PDBFixer

from openff.interchange import Interchange
from openff.interchange.components._packmol import UNIT_CUBE, pack_box, RHOMBIC_DODECAHEDRON

from openmm.openmm import System
from openmm import MonteCarloBarostat
from openmm import unit as openmm_unit
from openmm.app import PDBFile

from openmm.app.simulation import Simulation

import warnings
warnings.filterwarnings('ignore')

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!


In [2]:
tau_path = 'Inputs/8FUG_tau.pdb'

In [3]:
nglview.show_structure_file(tau_path)

NGLWidget()

In [4]:
from openbabel import openbabel
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("pdb", "mol")
 
mol = openbabel.OBMol()
obConversion.ReadFile(mol, tau_path)   # Open Babel will uncompress automatically
 
print(mol.NumAtoms())
print(mol.NumBonds())
print(mol.NumResidues())
 
obConversion.WriteFile(mol, 'Inputs/tau_mol.mol')

27070
27346
1702


True

In [5]:
tau = Molecule.from_file('Inputs/tau_mol.mol')

In [6]:
fixer = PDBFixer(tau_path)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
numChains = len(list(fixer.topology.chains()))
fixer.removeChains(range(4, numChains))
fixer.addSolvent(openmm.vec3.Vec3(15, 15, 15)*openmm.unit.nanometer)
PDBFile.writeFile(fixer.topology, fixer.positions, open("receptor_solvated.pdb", 'w'))

In [11]:
#Reimport to topology
top = Topology.from_pdb("receptor_solvated.pdb", unique_molecules=[tau, Molecule.from_smiles('[H]O[H]')])

In [12]:
sage_ff14sb = ForceField("openff-2.1.0.offxml", "ff14sb_off_impropers_0.0.3.offxml")
interchange = sage_ff14sb.create_interchange(top)

In [13]:
interchange.visualize()

NGLWidget()

### Equilibration

In [14]:
# Length of the simulation.
num_steps = 10000  # number of integration steps to run

# Logging options.
trj_freq = 10  # number of steps per written trajectory frame
data_freq = 10  # number of steps per written simulation statistics

# Integration options
time_step = 3 * openmm.unit.femtoseconds  # simulation timestep
temperature = 300 * openmm.unit.kelvin  # simulation temperature
friction = 1 / openmm.unit.picosecond  # friction constant

integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Create simulation from interchange file
simulation = interchange.to_openmm_simulation(integrator=integrator)
simulation.context.setVelocitiesToTemperature(temperature)
state = simulation.context.getState(getPositions=True, getEnergy=True)
print(state.getPotentialEnergy())
simulation.minimizeEnergy()

-4360938.222568337 kJ/mol


In [15]:
state = simulation.context.getState(getPositions=True, getEnergy=True)
state.getPotentialEnergy()

Quantity(value=-5537126.222568337, unit=kilojoule/mole)

In [16]:
# Add reporters
pdb_reporter = openmm.app.PDBReporter("trajectory_pdb.pdb", trj_freq)
dcd_reporter = openmm.app.DCDReporter("trajectory_dcd.dcd", trj_freq)

state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    reportInterval=5,
    step = True,             # writes the step number to each line
    time = True,             # writes the time (in ps)
    potentialEnergy = True,  # writes potential energy of the system (KJ/mole)
    kineticEnergy = True,    # writes the kinetic energy of the system (KJ/mole)
    totalEnergy = True,      # writes the total energy of the system (KJ/mole)
    temperature = True,      # writes the temperature (in K)
    volume = True,           # writes the volume (in nm^3)
    density = True)         # writes the density (in g/mL)

# Append state reporters
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(state_data_reporter)

#Simulation
print("Starting simulation...")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

# save the equilibration results to file
simulation.saveState('eq.state')
simulation.saveCheckpoint('eq.chk')

end = time.process_time()
print(f"Elapsed time {end - start} seconds")

print('Performed simulation')

Starting simulation...


OSError: [Errno 28] No space left on device

In [ ]:
nglview.show_mdtraj(md.load("trajectory_pdb.pdb"))